In [1]:
'''
Auto apply changes to docx from xlsx

this one is designed for 3rd round 2021

this one is designed for 4rd round 2022
'''
import csv
import docx
from tqdm import tqdm
import utils
import xlrd

In [3]:
# config
xlsx_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第四次会议/ALL.xlsx'

# in_path = '/home/gray/desktop/STANDARD/test2.docx' # example
in_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第四次会议/4.7.docx'

out_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第四次会议/out.docx'

doc = docx.Document(in_path)

In [8]:
replace = {}
workbook = xlrd.open_workbook(filename=xlsx_path)
worksheet = workbook.sheet_by_name(workbook.sheet_names()[0])

In [9]:
for i in tqdm(range(3, worksheet.nrows)):
    
    row = worksheet.row_values(i)
    
    # add auto-summit flag in row[0] so having increment all index
    sec_num = utils.get_sec_num_new(str(row[1]),from_head=True)
    pre_result = row[7]  # 0 -> deny, 1-> yes, 2->discuss, -1->auto
    
#     discuss = row[8]  # discuss flag
    
    if pre_result == '-1': # match case
        replace[row[2]] = row[3]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 525/525 [00:00<00:00, 399131.70it/s]


In [10]:
print(len(replace))
print(list(replace.items())[:2])

144
[('使用外部存储或是本地与外部混合的存储的系统称为存储虚拟器。', '存储虚拟器是指外部存储系统或者本地和外部混合的存储系统。'), ('健康和故障管理注意事项在参考的配置文件和包中被描述', '健康和故障管理注意事项在参考的配置文件和包中描述。')]


In [11]:
# replace没有区分正文和表格，做个safety check
track_replace = {}
for k in replace.keys():
    track_replace[k] = 0

In [12]:
# replace in text paragraphs
for para in tqdm(list(doc.paragraphs)):
    for a,b in replace.items():
        if a != '' and a in para.text:
            utils.replace_para_text(para,a,b)
            track_replace[a] += 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11872/11872 [00:29<00:00, 396.99it/s]


In [13]:
# replace in tables
for table_index in tqdm(range(len(list(doc.tables)))):
    table = doc.tables[table_index]
    for row in table.rows:
        for cell in row.cells:
            for para in cell.paragraphs:
                for a,b in replace.items():
                    if a!='' and a in para.text: # do replace
                        utils.replace_para_text(para,a,b)
                        track_replace[a] += 1

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 931/931 [01:04<00:00, 14.33it/s]


In [14]:
# check 0
for k,v in track_replace.items():
    if v != 1:
        print(k,v)
        print('--------------------')

"创建一个具体的StoragePool时，其存储容量直接或间接地从一个原始StoragePool获取。 0
--------------------
StoragePool实例提供了在给定StorageSetting作为一个目标的前提下，获取创建或者修改StorageVolume或者LogicalDisk时的可能大小的能力， 0
--------------------
用法值含义 2
--------------------
被删除状态 0
--------------------
(最后三段在英文版中没有找到对应？) 0
--------------------
InitialReplicationState是“Idle”值。 0
--------------------
一个灭有设备映射和屏蔽服务 0
--------------------
 0
--------------------
联合目标元素复制到源元素。 2
--------------------
一个组成池的能力会减弱通过提供引用给别人，但不是全部，只是对现有存储池的分量， 0
--------------------


In [15]:
len(track_replace)

144

In [16]:
def is_zero(n):
    return n == 0

tmp_zero = list(filter(is_zero,[v for v in track_replace.values()]))
print(len(tmp_zero))

8


In [17]:
doc.save(out_path)